# Rail Following - Live demo

### Load Trained Model

### Creating the Pre-Processing Function

In [1]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy.pth'))
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import time

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        print('heartbeat connection lost')
        while camera_links: camera_links.pop().unlink()
        while cont_links: cont_links.pop().unlink()
        robot.stop()

heartbeat = Heartbeat(period=5)
heartbeat.observe(handle_heartbeat_status, names='status') # attach the callback function to heartbeat status

print('ok')

ok


### Create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [17]:
angle = 0.0
angle_last = 0.0

def stop():
    camera.unobserve(execute, names='value')
    time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
    robot.stop()
    
def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0

    x_slider.value = x
    y_slider.value = y

    speed_slider.value = speed_gain_slider.value

    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = pid + steering_bias_slider.value

    reduction = 0.5
    
    left_motor = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0) * 0.5
    right_motor = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0) * 0.5
    left_slider.value = left_motor
    right_slider.value = right_motor
    
    if left_motor > right_motor
    
    
    #robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    #robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

    #robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, -0.33), 0.33)
    #robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, -0.33), 0.33)
    

    
def run():
    execute({'new': camera.value})
    camera.observe(execute, names='value')
    
print('ok')

ok


### Display Camera

### Create our robot instance which we'll need to drive the motors.

### Define sliders to control JetBot

### Display some sliders that will let us see what JetBot is thinking.

In [18]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets

#camera = Camera()
camera = Camera.instance(width=224, height=224, fps=10)
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
#display(image_widget)

from jetbot import Robot
robot = Robot()

speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.4, step=0.001, value=0.33, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.4, step=0.001, value=0.33, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.4, step=0.001, value=0.33, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.1, max=0.4, step=0.001, value=0.01, description='steering bias')
#1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
#2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
#3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

#display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

button_layout = ipywidgets.Layout(width='128px', height='64px')
stop_button = ipywidgets.Button(description='STOP', button_style='danger', layout=button_layout)
stop_button.on_click(lambda x: stop())
run_button = ipywidgets.Button(description='RUN', button_style='success', layout=button_layout)
run_button.on_click(lambda x: run())

sliders = widgets.VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider,run_button,stop_button])
display(widgets.HBox([image_widget,sliders]))

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

left_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='left')
right_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='right')

#The x and y sliders will display the predicted x, y values.
#The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
#nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

display(ipywidgets.HBox([y_slider, speed_slider,left_slider,right_slider]))
display(x_slider, steering_slider)

#try: run()
#except: pass

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

### Stop

In [4]:
try: stop()
except: pass